<a href="https://colab.research.google.com/github/DaverArenas/StockExchange_Predictions/blob/main/Random_Forest_With_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on this submission https://www.kaggle.com/code/listeningtounity/random-forest-with-optuna

In [3]:
import numpy as np
import pandas as pd

##Stock prices from Supplemental files 

In [1]:
path = "/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/supplemental_files/stock_prices.csv"

In [16]:
prices = pd.read_csv(path)

In [5]:
def prep_prices(prices):
    prices.Date = pd.to_datetime(prices.Date).view(int)
    prices["Volume"].fillna(1,inplace=True)
    prices.fillna(0,inplace=True)
    return prices

In [6]:
prices = prep_prices(prices)
feats = ["Date","SecuritiesCode","Open","High","Low","Close","Volume"]

In [7]:
prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,1638748800000000000,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,0.0,False,-0.003263
1,20211206_1332,1638748800000000000,1332,592.0,599.0,588.0,589.0,1360800,1.0,0.0,False,-0.008993
2,20211206_1333,1638748800000000000,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,0.0,False,-0.009963
3,20211206_1375,1638748800000000000,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,0.0,False,-0.015032
4,20211206_1376,1638748800000000000,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,0.0,False,0.002867


##Cantidad de acciones en el dataset

In [8]:
prices['SecuritiesCode'].nunique()

2000

##Split Data

In [26]:
X= prices[feats]
y = prices["Target"]

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [31]:
X_test = X_test.drop('Date', axis=1)

In [34]:
type(X_test['Volume'])

pandas.core.series.Series

##Using Optuna

In [11]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 3.2 MB/s 
     |████████████████████████████████| 209 kB 52.2 MB/s 
     |████████████████████████████████| 81 kB 9.2 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 112 kB 44.4 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 147 kB 74.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=c38f176014fd5b0f4c1c3fbf1bd87b58bd9a2792392a919e05938221e1ad3b2f
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import optuna

In [13]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 1000)
    
    regr = RandomForestRegressor(max_depth = max_depth, n_jobs=2)
    score = cross_val_score(regr, X, y, cv=5, scoring="r2")
    r2_mean = score.mean()
    return r2_mean

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout = 600)

model = RandomForestRegressor(max_depth = study.best_params['max_depth'], n_jobs=2)
model.fit(X,y)

[I 2022-08-04 18:00:03,493] A new study created in memory with name: no-name-28f73af0-e504-4ae6-aff4-a0973d9fb465
[I 2022-08-04 18:14:11,458] Trial 0 finished with value: -0.6245524433861647 and parameters: {'max_depth': 319}. Best is trial 0 with value: -0.6245524433861647.


RandomForestRegressor(max_depth=319, n_jobs=2)

In [33]:
y_pred = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Date

  warnings.warn(message, FutureWarning)


ValueError: ignored

In [17]:
prices_train = pd.read_csv("/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/train_files/stock_prices.csv")

In [21]:
# Drop Date column from multi_model
single_model = prices_train.drop('Date', axis=1)

In [22]:
# create Train X and y df's
X = single_model.drop('Target', axis=1)
y = single_model['Target']

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=50)

In [25]:
X_test

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
1758101,20201002_6430,6430,1010.0,1015.0,991.0,994.0,42100,1.0,NaN,False
1331548,20191112_3443,3443,7250.0,7380.0,7160.0,7310.0,15600,1.0,NaN,False
5608,20170110_1515,1515,5510.0,5560.0,5450.0,5510.0,16600,1.0,NaN,False
1115308,20190530_2335,2335,954.0,960.0,917.0,925.0,62200,1.0,NaN,False
976658,20190207_5911,5911,1969.0,1970.0,1919.0,1922.0,96900,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...
1396505,20191227_4776,4776,1419.0,1495.0,1395.0,1474.0,1278600,1.0,NaN,False
1274119,20190926_8095,8095,498.0,498.0,489.0,493.0,118100,1.0,NaN,False
1815838,20201113_6470,6470,611.0,611.0,578.0,585.0,55200,1.0,NaN,False
2585,20170105_4914,4914,3160.0,3160.0,3095.0,3125.0,14600,1.0,NaN,False


In [24]:
y_pred = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- AdjustmentFactor
- ExpectedDividend
- RowId
- SupervisionFlag
Feature names seen at fit time, yet now missing:
- Date

  warnings.warn(message, FutureWarning)


ValueError: ignored